# Structure Property visualizer

This is the second of two deliverables for the SiSc-Lab2020 project.

Authors = 

Supervisors: Dr. Jens Bröder, Dr. Daniel Wortmann, Johannes Wasmer, Prof. Dr. Stefan Blügel.

# Instructions by supervisors

## Jens

You have to implement this notebook

In the end only Markup text and output results should be seen if one hides the code cells (hide_code extension)

That is can easily exported into a nice pdf file.

Also the amount of python code in this notebook should be minimal.

Rather export the functions you use from a different file.

optional dump query results in a file, from which results will be reread for speed, i.e cache results

______________

An example of what we in the end aim for your find here.

https://www.materialscloud.org/discover/mofs#mcloudHeader

Clicking at one of the prerendered plots will open an interactive bokeh app.

Code for this you find here: https://github.com/materialscloud-org/structure-property-visualizer/blob/master/figure/main.py

For you sisc project this is to much, so do not view this as something you have to deliver but something that with the help of your work
in this project we will build out of it for our purposes.

So do not worry about an app.

_________________

**Your tasks are as follows:**

1. Implement a general interactive bokeh scatter plots with linked histograms (see static version examples/MP_convergence_scf_clean_150_240.png). 

   We gladly help you with it. (this goes into helpers.py and you import it here for usage). This function should not contain any aiida methods.
2. Extract float data you find in certain 'Dict' nodes into a/several pandas object, which will be then the datasource for this notebook. 

   So this notebook should not directly depend on any aiida methods (beyond load_profile)


## Johannes

Nothing much to add.

I'd just have bonus: if all this works, one could think about some actual data analysis, like clustering analysis or dimension reduction as we learned in Data Analysis & Visualization. For example, do [PCA](https://blog.exploratory.io/an-introduction-to-principal-component-analysis-pca-with-2018-world-soccer-players-data-810d84a14eab) with the quantities magnetic moment, energy, band gap, fermi energy, (structure, core levels). Ie, are some of these quantities (linearly) correlated in a data subset (e.g. all output nodes)? Since the data is already preprocessed mostly, this should be relatively easy, just throw the data into [scikit-learn](https://scikit-learn.org/stable/).

# Imports

In [2]:
# magics:
# # autoreload imports. 
# # intent: if i change sth in import, i don't have to restart kernel. enable only for development.
%load_ext autoreload
%autoreload 2
# # choose matplotlib backend. backend 'notebook' allows interactive plots if your env allows it.
%matplotlib notebook

In [3]:
# python imports:
from collections import Counter
import time
import numpy as np
import pandas as pd
#from pprint import pprint

In [ ]:
# aiida imports:
from aiida import load_profile
profile = load_profile()

# ggf add further imports

In [4]:
# project imports:
import helpers
# from .helpers import bokeh_struc_prop_vis
# from .helpers import generate_structure_property_pandas_source

# equivalent ('.' is the sisc_lab directory):
# from . import helpers
# alternative:
# from .helpers import print_bold
# from .helpers import * ('*' import everything; use of '*' is considered bad style)

In [5]:
# (example:)
helpers.print_bold(f"This notebook/dashboard will visualize the contents from the database of profile {profile.name}")

This notebook/dashboard will visualize the contents from the database of profile seconddb


# Subastk D2.a: Data acquisition

Task: check which output dict nodes returned by workflows, which had `StructureData` nodes as inputs are there in the database.

For example, for a successful `FleurScfWorkChain`, there are two return `Dict` nodes, one is linked with
`last_fleur_calc_output` and one with `output_scf_wc_para`.

If a `StructureData` is an input of such a workflow you can extract the formula, `uuid` and other information you need from the 
`StructureData` which is always linked into workflows via the link name `structure`.

All the user should have to say:
```python
source = generate_structure_property_pandas_source('<workflow_name>')
```


In [1]:
#Find out valid plugin tools
# !verdi plugin list aiida.workflows

#Have a look at the attributes of dict nodes and sturcture nodes
# dict_project=['attributes','uuid']
# workflowdictlst = helpers.get_structure_workflow_dict(dict_project=dict_project)
#or
# structure_project=['uuid', 'extras','attributes']
# workflowdictlst = helpers.get_structure_workflow_dict(structure_project=structure_project)

In [7]:
# workflow_name = 'fleur_scf_wc' # Filter workflow
# workflowdictlst = helpers.get_structure_workflow_dict(workflow_filters={'attributes.process_label':workflow_name})
#or
workflow_name = None # No restriction. Querying by default
workflowdictlst = helpers.get_structure_workflow_dict()

print(len(workflowdictlst))
workflowdictlst[:2]

850


[{'structure': ['02e6640d-d7cc-48ab-b5fc-403f6c5f747d', None],
  'workflow': ['de3b637c-138f-41f0-aada-21ded6bb520b', 'FleurScfWorkChain'],
  'dict': ['d8cab742-c2a1-48a6-9026-1136e80cccaa']},
 {'structure': ['02e6640d-d7cc-48ab-b5fc-403f6c5f747d', None],
  'workflow': ['de3b637c-138f-41f0-aada-21ded6bb520b', 'FleurScfWorkChain'],
  'dict': ['98265865-e402-4349-a731-62c94c1cc349']}]

In [8]:
# Some dict attributes: force, energy, total_energy, fermi_energy, bandgap, charge_density, distance_charge, 
#                       workflow_version, parser_info, material,
# Remarks:  workflow_version=0.2.2: energy, force
#           workflow_version=0.4.2: total_energy, material. 
#           parser: fermi_energy, bandgap.

dict_project=['uuid', 'attributes.force', 'attributes.energy', 'attributes.total_energy', 'attributes.distance_charge',             'attributes.bandgap', 'attributes.workflow_version', 'attributes.parser_info', 'attributes.material',
        'attributes.fermi_energy']
dictpd = helpers.generate_dict_property_pandas_source(
            workflow_name, 
            dict_project=dict_project, 
            filename='dict_property.json')
dictpd.head(5)

,dict_uuid,force,energy,total_energy,distance_charge,bandgap,workflow_version,parser_info,material,fermi_energy
0,d8cab742,None,NaN,-2545.579023,0.000049,NaN,0.4.2,None,Fe2,NaN
1,98265865,None,-69268.734001,NaN,NaN,0.007221,None,AiiDA Fleur Parser v0.3.1,None,0.353498
2,93bbe25d,None,NaN,-971.290635,NaN,NaN,0.4.2,None,Al4,NaN
3,1e42c371,None,-26430.164420,NaN,NaN,0.163845,None,AiiDA Fleur Parser v0.3.0,None,0.299326
4,5bc3c4d9,[0.0],-26430.080872,NaN,NaN,NaN,0.2.2,None,None,NaN


In [9]:
structure_project=['uuid', 'extras.formula']
structurepd = helpers.generate_structure_property_pandas_source(
            workflow_name, 
            structure_project=structure_project,
            filename='structure_property.json')
structurepd.head(5)

,structure_uuid,formula
0,02e6640d,None
1,02e6640d,None
2,03bc06be,None
3,03bc06be,None
4,03bc06be,None


In [10]:
# Generate the combination of the above
combinepd = helpers.generate_combination_property_pandas_source(
            workflow_name, 
            dict_project=dict_project, 
            structure_project=structure_project,
            filename='combined_property.json')
combinepd.head(5)

,dict_uuid,force,energy,total_energy,distance_charge,bandgap,workflow_version,parser_info,material,fermi_energy,structure_uuid,formula
0,d8cab742,None,NaN,-2545.579023,0.000049,NaN,0.4.2,None,Fe2,NaN,02e6640d,None
1,98265865,None,-69268.734001,NaN,NaN,0.007221,None,AiiDA Fleur Parser v0.3.1,None,0.353498,02e6640d,None
2,93bbe25d,None,NaN,-971.290635,NaN,NaN,0.4.2,None,Al4,NaN,03bc06be,None
3,1e42c371,None,-26430.164420,NaN,NaN,0.163845,None,AiiDA Fleur Parser v0.3.0,None,0.299326,03bc06be,None
4,5bc3c4d9,[0.0],-26430.080872,NaN,NaN,NaN,0.2.2,None,None,NaN,03bc06be,None


# Subtask D2.b: Interactive plot

Allow the user to choose, which properties to plot on what axis.

```python
xdata = source['distance']
ydata = source['energy']
```

Single bokeh scatter plot with histpgrams on both sides, hover tool should show 'input structure, formula, 
structure_uuid and dictnode uuid', as long as this information is available.

```python
bokeh_struc_prop_vis(xdata, ydata, src=source, **kwargs)
```

In [80]:
from helpers import read_json_file
df, xdata, ydata = read_json_file('combined_property.json','energy', 'bandgap')
df

,dict_uuid,force,energy,total_energy,distance_charge,bandgap,workflow_version,parser_info,material,fermi_energy,structure_uuid,formula
0,98265865,None,-69268.734001,NaN,NaN,7.221356e-03,None,AiiDA Fleur Parser v0.3.1,None,0.353498,02e6640d,NaN
1,1e42c371,None,-26430.164420,NaN,NaN,1.638455e-01,None,AiiDA Fleur Parser v0.3.0,None,0.299326,03bc06be,NaN
2,07610300,None,-26430.178902,NaN,NaN,5.601285e-03,None,AiiDA Fleur Parser v0.3.0,None,0.292627,04b4e5aa,NaN
3,ddf9ee72,None,-26430.097504,NaN,NaN,6.200935e-03,None,AiiDA Fleur Parser v0.3.0,None,0.329099,055e8c97,NaN
4,acfc924c,None,-69269.612046,NaN,NaN,1.832472e-04,None,AiiDA Fleur Parser v0.3.0,None,0.379085,09b7450c,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
283,e6ecbc0f,None,-69268.638764,NaN,NaN,1.464338e-02,None,AiiDA Fleur Parser v0.3.1,None,0.399115,f540a37f,NaN
284,bba5ea0a,None,-26430.091615,NaN,NaN,1.140000e-08,None,AiiDA Fleur Parser v0.3.0,None,0.334367,f67d62d7,NaN
285,9142422f,None,-69269.472305,NaN,NaN,9.668811e-04,None,AiiDA Fleur Parser v0.3.0,None,0.359201,f999a276,NaN
286,984358bd,None,-15784.731259,NaN,NaN,9.299812e-01,None,AiiDA Fleur Parser v0.3.0,None,0.193149,fb3d7bd9,NaN


In [83]:
from helpers import bokeh_struc_prop_vis
bokeh_struc_prop_vis('combined_property.json', 'energy', 'bandgap')